In [2]:
from math import log2
x = [3, 1, 7, 0, 4, 1, 6, 3]
y = [0 for _ in x]

![H2](h2.png)

In [5]:
def scan(x, n):
    xy = [_ for _ in x]
    print(xy)
    for d in range( int(log2(n))):
        print('d: ', d)
        for i in range(0, n, int(2**(d+1))):
            i1 = i + 2**d -1
            i2 = i + 2**(d+1) -1 
            xy[i2] = xy[i1] + xy[i2]
            
            # print('i = ', i, ': i + 2**d -1 = ', i1)
            # print('i = ', i, ': i + 2**(d+1) -1 = ', i2)        
    return xy

expected = [3, 4, 7, 11, 4, 5, 6, 25]
xy = scan(x, len(x))
print(f'xy == {expected}', xy == expected)

[3, 1, 7, 0, 4, 1, 6, 3]
d:  0
d:  1
d:  2
xy == [3, 4, 7, 11, 4, 5, 6, 25] True


In [6]:
def down_by_half(n):
    num = n >> 1
    while num > 0:        
        yield num
        num >>= 1

def up_by_half(n):
    num = 1
    while num < n:
        yield num
        num <<= 1

![H3](h3.png)

![H4](h4.png)

In [7]:
def scan(x, y, n):        
    xy = [0 for _ in x]

    # copy input values from device memory x to shared memory xy
    for tid in range(n>>1):
        xy[2*tid] = x[2*tid]
        xy[2*tid+1] = x[2*tid+1]            
    print(f'xy: {xy}')

    # build sums in place up the tree
    stride = 1
    for d in down_by_half(n):
        for tid in range(d):
            ai = stride * (2*tid + 1) -1
            bi = stride * (2*tid + 2) -1
            xy[bi] += xy[ai]
        print(xy)
        stride *= 2

    # clear last element
    print('--'*20)
    xy[n-1] = 0
    print(xy)

    # traverse down tree & build scan
    for d in up_by_half(n):
        stride >>=1
        for tid in range(d):
            ai = stride * (2*tid + 1) -1
            bi = stride * (2*tid + 2) -1
            temp = xy[ai]
            xy[ai] = xy[bi]            
            xy[bi] += temp
        print(xy)
            
    # write results to device memory y
    for tid in range(n>>1):
        y[2*tid] = xy[2*tid]
        y[2*tid+1] = xy[2*tid+1]            
    

scan(x, y, len(x))
expected = [0, 3, 4, 11, 11, 15, 16, 22]
print(f'y == {expected}', y == expected)

xy: [3, 1, 7, 0, 4, 1, 6, 3]
[3, 4, 7, 7, 4, 5, 6, 9]
[3, 4, 7, 11, 4, 5, 6, 14]
[3, 4, 7, 11, 4, 5, 6, 25]
----------------------------------------
[3, 4, 7, 11, 4, 5, 6, 0]
[3, 4, 7, 0, 4, 5, 6, 11]
[3, 0, 7, 4, 4, 11, 6, 16]
[0, 3, 4, 11, 11, 15, 16, 22]
y == [0, 3, 4, 11, 11, 15, 16, 22] True
